In [1]:
# Import the Pandas library

import openpyxl
import io
import tempfile
import pandas as pd
import numpy as np
import sys

sys.path.append('../')

from models.linear_regression import LinReg

lin_reg = LinReg()
path = "../.test_data/more_data.xlsx"

coded_df = lin_reg.process_excel(path)


print(coded_df.info())
coded_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41990 entries, 0 to 41989
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NAME                 41990 non-null  object 
 1   date                 41990 non-null  int64  
 2   color_and_value      41990 non-null  object 
 3   name_code            41990 non-null  int64  
 4   day_of_month         41990 non-null  int32  
 5   day_of_week          41990 non-null  int32  
 6   date_code            41990 non-null  int64  
 7   day_of_year          41990 non-null  int64  
 8   value                41990 non-null  float64
 9   color                41990 non-null  object 
 10  extra                41990 non-null  float64
 11  color_code           41990 non-null  int64  
 12  next_color_code      41860 non-null  float64
 13  previous_color_code  41860 non-null  float64
 14  color_binary         41990 non-null  int64  
 15  next_color_binary    41860 non-null 

,NAME,date,color_and_value,name_code,day_of_month,day_of_week,date_code,day_of_year,value,color,extra,color_code,next_color_code,previous_color_code,color_binary,next_color_binary
0,GJ1,1735257600000000000,28.26107 | A9D08E | 214257094.0,1,27,4,0,362,28.261070,A9D08E,214257094.0,0,3.0,NaN,0,0.0
1,GJ2,1735257600000000000,47.95346 | A9D08E | 229631139.0,2,27,4,0,362,47.953460,A9D08E,229631139.0,0,3.0,NaN,0,0.0
2,GJ3,1735257600000000000,78.28529 | 9BC2E6 | 1804911.0,3,27,4,0,362,78.285290,9BC2E6,1804911.0,1,0.0,NaN,1,0.0
3,GJ4,1735257600000000000,47.75716 | A9D08E | 4872234.0,4,27,4,0,362,47.757160,A9D08E,4872234.0,0,3.0,NaN,0,0.0
4,GJ5,1735257600000000000,16.25788 | A9D08E | 226485.0,5,27,4,0,362,16.257880,A9D08E,226485.0,0,1.0,NaN,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41985,GJ132,1713398400100000000,8.889043 | FFC000 | 160498,132,18,3,322,109,8.889043,FFC000,160498.0,2,NaN,0.0,1,NaN
41986,GJ133,1713398400100000000,-0.02956 | A9D08E | 1757606,133,18,3,322,109,-0.029560,A9D08E,1757606.0,0,NaN,0.0,0,NaN
41987,GJ134,1713398400100000000,22.86888 | FFC000 | 230548,134,18,3,322,109,22.868880,FFC000,230548.0,2,NaN,0.0,1,NaN
41988,GJ135,1713398400100000000,53.2374 | 8EA9DB | 2633516,135,18,3,322,109,53.237400,8EA9DB,2633516.0,6,NaN,2.0,1,NaN


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



# Splitting the dataset into the Training set and Test set
X , y = lin_reg.train_test_split(coded_df)


X.shape, y.shape

((41730, 8), (41730,))

In [3]:
# Create and train the model


lin_reg.fit(X, y)

print(f'mse : {lin_reg.mse}' )
print(f'accuracy: {lin_reg.accuracy}' )
print(f'preferred_accuracy: {lin_reg.preferred_accuracy}' )

print(f'predictions_count: {lin_reg.predictions_count}   in {lin_reg.y_test_count}' )


prob = coded_df[coded_df['color_binary'] == 1].shape[0] / coded_df.shape[0]

print(f'real prob: {prob}' )


mse : 0.5008387251377906
accuracy: 0.49916127486220946
preferred_accuracy: 0.6601941747572816
predictions_count: 1339   in 4609
real prob: 0.5566087163610384


In [4]:
# make predictions
df = lin_reg.predict_last()

df

,NAME,2024-04-18 00:00:00.1,color_and_value,next_color_code
0,GJ1,-1.66649 | D5A6BD | 317267066,-1.66649 | D5A6BD | 317267066,0.681941
1,GJ2,-5.91249 | FFC000 | 186069576,-5.91249 | FFC000 | 186069576,0.667174
2,GJ3,11.11636 | FFC000 | 1134184,11.11636 | FFC000 | 1134184,0.668339
3,GJ4,0.568637 | FFC000 | 2696593,0.568637 | FFC000 | 2696593,0.668586
4,GJ5,16.59896 | D5A6BD | 290190,16.59896 | D5A6BD | 290190,0.684565
...,...,...,...,...
125,GJ132,8.889043 | FFC000 | 160498,8.889043 | FFC000 | 160498,0.705974
126,GJ133,-0.02956 | A9D08E | 1757606,-0.02956 | A9D08E | 1757606,0.690608
127,GJ134,22.86888 | FFC000 | 230548,22.86888 | FFC000 | 230548,0.706608
128,GJ135,53.2374 | 8EA9DB | 2633516,53.2374 | 8EA9DB | 2633516,0.738237


In [5]:
print(df[(df['next_color_code'] != 0)  ].shape)
df[(df['next_color_code'] != 0) ]
df.sort_values(by='next_color_code', ascending=False)[:20]

(130, 4)


,NAME,2024-04-18 00:00:00.1,color_and_value,next_color_code
128,GJ135,53.2374 | 8EA9DB | 2633516,53.2374 | 8EA9DB | 2633516,0.738237
122,GJ129,27.78279 | 8EA9DB | 719341,27.78279 | 8EA9DB | 719341,0.736403
116,GJ122,63.47085 | 8EA9DB | 5135358,63.47085 | 8EA9DB | 5135358,0.734470
114,GJ120,14.6452 | 8EA9DB | 7918449,14.6452 | 8EA9DB | 7918449,0.733698
113,GJ119,1.145039 | 8EA9DB | 46456733,1.145039 | 8EA9DB | 46456733,0.733188
112,GJ118,16.95284 | 8EA9DB | 3760380,16.95284 | 8EA9DB | 3760380,0.733141
110,GJ116,9.420818 | 8EA9DB | 672758,9.420818 | 8EA9DB | 672758,0.732544
108,GJ112,21.23892 | 8EA9DB | 7108736,21.23892 | 8EA9DB | 7108736,0.731391
106,GJ110,17.16026 | 8EA9DB | 1467286,17.16026 | 8EA9DB | 1467286,0.730818
103,GJ107,43.65911 | 8EA9DB | 4855479,43.65911 | 8EA9DB | 4855479,0.730023
